In [1]:
import s3fs
import os
import pandas as pd
from collections import OrderedDict
from sqlalchemy import create_engine, inspect
from zipfile import ZipFile


In [2]:
fs = s3fs.S3FileSystem(anon=True)
fs.ls('tripdata')


['tripdata/201306-citibike-tripdata.zip',
 'tripdata/201307-201402-citibike-tripdata.zip',
 'tripdata/201307-citibike-tripdata.zip',
 'tripdata/201308-citibike-tripdata.zip',
 'tripdata/201309-citibike-tripdata.zip',
 'tripdata/201310-citibike-tripdata.zip',
 'tripdata/201311-citibike-tripdata.zip',
 'tripdata/201312-citibike-tripdata.zip',
 'tripdata/201401-citibike-tripdata.zip',
 'tripdata/201402-citibike-tripdata.zip',
 'tripdata/201403-citibike-tripdata.zip',
 'tripdata/201404-citibike-tripdata.zip',
 'tripdata/201405-citibike-tripdata.zip',
 'tripdata/201406-citibike-tripdata.zip',
 'tripdata/201407-citibike-tripdata.zip',
 'tripdata/201408-citibike-tripdata.zip',
 'tripdata/201409-citibike-tripdata.zip',
 'tripdata/201410-citibike-tripdata.zip',
 'tripdata/201411-citibike-tripdata.zip',
 'tripdata/201412-citibike-tripdata.zip',
 'tripdata/201501-citibike-tripdata.zip',
 'tripdata/201502-citibike-tripdata.zip',
 'tripdata/201503-citibike-tripdata.zip',
 'tripdata/201504-citibike-

In [3]:
for _file in fs.ls('tripdata'):
    if not _file.endswith('.zip'):
        continue
    final_df = pd.DataFrame()
    fp = fs.open(_file)
    with ZipFile(fp) as zip:
        for index, member in enumerate(zip.namelist()):
            if "__MAC" in member:
                continue
            csv = zip.open(member)
            member_df = pd.read_csv(csv)
            final_df = pd.concat([final_df, member_df], sort=False)
final_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,142,2019-02-01 15:35:02.0820,2019-02-01 15:37:24.1360,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,29677,Subscriber,1963,1
1,223,2019-02-01 17:00:46.8900,2019-02-01 17:04:30.5500,3183,Exchange Place,40.716247,-74.033459,3681,Grand St,40.715178,-74.037683,26234,Subscriber,1992,2
2,106,2019-02-01 17:08:01.3260,2019-02-01 17:09:47.4400,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,29588,Subscriber,1960,1
3,370,2019-02-01 17:09:31.2100,2019-02-01 17:15:41.6550,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,29250,Subscriber,1976,1
4,315,2019-02-01 17:19:53.2490,2019-02-01 17:25:09.1400,3183,Exchange Place,40.716247,-74.033459,3273,Manila & 1st,40.721651,-74.042884,29586,Subscriber,1980,1


In [4]:
OUTPUT = os.path.join('..', 'temp_output', 'combined.csv')
final_df.to_csv(OUTPUT, index=False)